In [1]:
import requests
import csv
from datetime import datetime, timedelta
import time
import pandas as pd

## Kijkcijfer data ophalen van laatste 10 jaar
Indien data mist wordt deze genegeerd. Alle andere data wordt volledig in een CSV file gestoken die later gefilterd en gecleaned kan worden

In [2]:
start_date = datetime(2025, 2, 26)
end_date = datetime.today()

# CSV-bestand aanmaken
with open('./data/raw/tv_kijkcijfers_raw_TEST.csv', 'w') as csvfile:
  fieldnames = [
    'dateDiff', 'ranking', 'description', 'channel', 
    'startTime', 'rLength', 'rateInK', 'live'
  ]
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  writer.writeheader()

  # Loop door elke dag
  current_date = start_date
  while current_date <= end_date:
    datum = f"{current_date.year}-{current_date.month}-{current_date.day}"
    url = f"https://api.cim.be/api/cim_tv_public_results_daily_views?dateDiff={datum}&reportType=north"
        
    try:
      response = requests.get(url)
      if response.status_code == 200:
        data = response.json()
        programma_lijst = data.get('hydra:member', [])
                
        for programma in programma_lijst:
          try:
            writer.writerow({
              'dateDiff': programma.get('dateDiff'),
              'ranking': programma.get('ranking'),
              'description': programma.get('description'),
              'channel': programma.get('channel'),
              'startTime': programma.get('startTime'),
              'rLength': programma.get('rLength'),
              'rateInK': programma.get('rateInK'),
              'live': programma.get('live')
            })
                        
          except Exception as e:
            print(f"Fout bij verwerken programma op {datum}: {e}")
                
        print(f"Success: {datum} ({len(programma_lijst)} programma's)")
      else:
        print(f"Geen data voor {datum} (HTTP {response.status_code})")
                
    except Exception as e:
      print(f"Fout bij ophalen {datum}: {e}")
        
    current_date += timedelta(days=1)
print("Alle data opgehaald!")

Success: 2025-2-26 (20 programma's)
Success: 2025-2-27 (20 programma's)
Success: 2025-2-28 (20 programma's)
Success: 2025-3-1 (20 programma's)
Success: 2025-3-2 (20 programma's)
Success: 2025-3-3 (20 programma's)
Success: 2025-3-4 (20 programma's)
Success: 2025-3-5 (20 programma's)
Success: 2025-3-6 (20 programma's)
Success: 2025-3-7 (20 programma's)
Success: 2025-3-8 (20 programma's)
Success: 2025-3-9 (20 programma's)
Success: 2025-3-10 (20 programma's)
Success: 2025-3-11 (20 programma's)
Success: 2025-3-12 (20 programma's)
Success: 2025-3-13 (20 programma's)
Success: 2025-3-14 (0 programma's)
Success: 2025-3-15 (0 programma's)
Alle data opgehaald!


# Weerdata van laatste 10 jaar ophalen

In [3]:
# Locatie voor Vlaanderen (Brussel als centraal punt)
latitude = 50.8503
longitude = 4.3517

# Datumberekening
start_date = datetime(2025, 2, 26)
end_date = datetime.today()

# API parameters - Relevante hourly variabelen voor een ML model voor kijkcijfers
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date.strftime('%Y-%m-%d'),
    "end_date": end_date.strftime('%Y-%m-%d'),
    "hourly": [
        "temperature_2m",   # Gemiddelde temperatuur per uur
        "weathercode",      # Weertype als code per uur
        "precipitation",    # Totale neerslag per uur
        "rain",             # Regen per uur
        "snowfall",         # Sneeuwval per uur
        "cloudcover",       # Bewolking per uur
        "windspeed_10m"     # Windsnelheid per uur
    ],
    "timezone": "Europe/Brussels",
    "temperature_unit": "celsius",
    "precipitation_unit": "mm",
    "windspeed_unit": "kmh"
}

# Data ophalen
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()["hourly"]
    
    # DataFrame maken met de opgehaalde variabelen
    df = pd.DataFrame({
        "time": data["time"],
        "temperature": data["temperature_2m"],
        "weather_code": data["weathercode"],
        "precipitation": data["precipitation"],
        "rain": data["rain"],
        "snowfall": data["snowfall"],
        "cloudcover": data["cloudcover"],
        "windspeed": data["windspeed_10m"]
    })
    
    # Converteer de tijdkolom naar datetime en extraheer temporele features
    df['time'] = pd.to_datetime(df['time'])
    df['hour'] = df['time'].dt.hour              # Uur van de dag
    df['day_of_week'] = df['time'].dt.dayofweek     # 0 = maandag, 6 = zondag
    df['month'] = df['time'].dt.month             # Maand (1 t/m 12)
    df['year'] = df['time'].dt.year               # Jaar
    
    # Opslaan als CSV-bestand (voor later gebruik in ML modellen)
    df.to_csv("./data/raw/weerdata_TEST.csv", index=False)
    print("Successvol opgeslagen als weerdata.csv")

else:
    print(f"Fout: {response.status_code}")
    print(response.text)

Successvol opgeslagen als weerdata.csv
